### Section 1: Redfin data wrangling
Turn the Redfin data into a limited dataset that has year, quarter, zip, and median sale price.
This data only goes back to 2012.

In [1]:
import pandas as pd
df = pd.read_csv("data/zip_code_market_tracker.tsv000", sep='\t')

In [2]:
df.head()

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2022-06-01,2022-08-31,90,zip code,2,30725,f,Zip Code: 71923,NaN,Arkansas,...,-0.090769,NaN,NaN,NaN,0.25,-0.016667,0.083333,"Arkadelphia, AR",11660,2023-04-09 14:55:08
1,2019-12-01,2020-02-29,90,zip code,2,21126,f,Zip Code: 49265,NaN,Michigan,...,-0.185294,NaN,NaN,NaN,0.20,0.200000,-0.300000,"Adrian, MI",10300,2023-04-09 14:55:08
2,2012-07-01,2012-09-30,90,zip code,2,34119,f,Zip Code: 78727,NaN,Texas,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,"Austin, TX",12420,2023-04-09 14:55:08
3,2012-11-01,2013-01-31,90,zip code,2,13119,f,Zip Code: 31503,NaN,Georgia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Waycross, GA",48180,2023-04-09 14:55:08
4,2016-04-01,2016-06-30,90,zip code,2,34063,f,Zip Code: 78648,NaN,Texas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Austin, TX",12420,2023-04-09 14:55:08


In [3]:
len(df)
min(df['period_end'])

'2012-03-31'

In [4]:
df.columns

Index(['period_begin', 'period_end', 'period_duration', 'region_type',
       'region_type_id', 'table_id', 'is_seasonally_adjusted', 'region',
       'city', 'state', 'state_code', 'property_type', 'property_type_id',
       'median_sale_price', 'median_sale_price_mom', 'median_sale_price_yoy',
       'median_list_price', 'median_list_price_mom', 'median_list_price_yoy',
       'median_ppsf', 'median_ppsf_mom', 'median_ppsf_yoy', 'median_list_ppsf',
       'median_list_ppsf_mom', 'median_list_ppsf_yoy', 'homes_sold',
       'homes_sold_mom', 'homes_sold_yoy', 'pending_sales',
       'pending_sales_mom', 'pending_sales_yoy', 'new_listings',
       'new_listings_mom', 'new_listings_yoy', 'inventory', 'inventory_mom',
       'inventory_yoy', 'months_of_supply', 'months_of_supply_mom',
       'months_of_supply_yoy', 'median_dom', 'median_dom_mom',
       'median_dom_yoy', 'avg_sale_to_list', 'avg_sale_to_list_mom',
       'avg_sale_to_list_yoy', 'sold_above_list', 'sold_above_list_mom',
 

In [5]:
df['property_type'].unique()

array(['All Residential', 'Townhouse', 'Single Family Residential',
       'Multi-Family (2-4 Unit)', 'Condo/Co-op'], dtype=object)

In [6]:
count = df['property_type'].value_counts()
count

All Residential              2318669
Single Family Residential    2274100
Condo/Co-op                   894931
Townhouse                     737934
Multi-Family (2-4 Unit)       686305
Name: property_type, dtype: int64

In [7]:
# create year and quarter columns from the dates
df['period_begin'] = pd.to_datetime(df['period_begin'])
df['period_end'] = pd.to_datetime(df['period_end'])
df['year'] = df['period_end'].dt.year
df['quarter'] = df['period_end'].dt.quarter

In [8]:
df['zip_code']= df['region'].str.split(':').str[-1].str.strip()
df['zip_code']=df['zip_code'].astype(float)

In [9]:
quarterly_prices_by_zip = df[['state', 'zip_code', 'year', 'quarter', 'median_sale_price', 'homes_sold', 'property_type', 'months_of_supply']]

In [10]:
# save off the narrowed redfin data
quarterly_prices_by_zip.to_csv('data/cleaned data/redfin_quarterly_prices_by_zip.csv', index = False)

In [11]:
quarterly_prices_by_zip.head()

,state,zip_code,year,quarter,median_sale_price,homes_sold,property_type,months_of_supply
0,Arkansas,71923.0,2022,3,167500.0,50.0,All Residential,NaN
1,Michigan,49265.0,2020,1,294000.0,20.0,All Residential,NaN
2,Texas,78727.0,2012,3,140000.0,1.0,Townhouse,NaN
3,Georgia,31503.0,2013,1,60000.0,1.0,Single Family Residential,NaN
4,Texas,78648.0,2016,2,229500.0,1.0,Multi-Family (2-4 Unit),NaN


In [12]:
# create the total value of property in each record (homes sold times price)
quarterly_prices_by_zip['total_sale_value_sum'] = quarterly_prices_by_zip['median_sale_price'] * quarterly_prices_by_zip['homes_sold']


/var/folders/hl/p3c4xh5x4nv00kvc7sckx1040000gn/T/ipykernel_72560/2800580599.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quarterly_prices_by_zip['total_sale_value_sum'] = quarterly_prices_by_zip['median_sale_price'] * quarterly_prices_by_zip['homes_sold']


# creating the yearly dataset we want
We will be dropping multi-family from the analysis and just looking at single family
We will be summing sales reported per quarter, but then taking a weighted median value based on the number of observations
This should give us a dataset that has zipcode, year, normalized sale price, and homes sold, that are not multifamily
Then we can merge this with the rest of the data and use these prices as baselines for estimating previous year prices off of HPI data

In [13]:
# dropping the multi-family
quarterly_single_family_by_zip = quarterly_prices_by_zip.drop(quarterly_prices_by_zip[quarterly_prices_by_zip['property_type'] == 'Multi-Family (2-4 Unit)'].index)
# calculating a normalized, aka weighted, estimate of the true media value: a weighted avg of the medians collected (sum of all the total values / all homes sold)
yearly_single_family_by_zip = quarterly_single_family_by_zip.groupby(['zip_code', 'year']).agg({'total_sale_value_sum':'sum', 'homes_sold': 'sum'}).reset_index()
yearly_single_family_by_zip['normalized_sale_price'] = yearly_single_family_by_zip['total_sale_value_sum']/yearly_single_family_by_zip['homes_sold']

In [14]:
yearly_single_family_by_zip.head()

,zip_code,year,total_sale_value_sum,homes_sold,normalized_sale_price
0,501.0,2012,1080000.0,6.0,180000.0
1,501.0,2013,1530000.0,6.0,255000.0
2,501.0,2015,2465240.0,10.0,246524.0
3,501.0,2016,412000.0,2.0,206000.0
4,501.0,2017,621468.0,4.0,155367.0


In [15]:
# checking what this looks like in my zip code
test = yearly_single_family_by_zip[yearly_single_family_by_zip['zip_code']==97405]
test

,zip_code,year,total_sale_value_sum,homes_sold,normalized_sale_price
230360,97405.0,2012,6.892417e+08,2953.0,233403.902811
230361,97405.0,2013,1.015211e+09,3882.0,261517.534261
230362,97405.0,2014,1.006331e+09,3763.0,267427.814510
230363,97405.0,2015,1.286704e+09,4750.0,270885.071158
230364,97405.0,2016,1.493344e+09,5086.0,293618.525757
230365,97405.0,2017,1.540981e+09,4948.0,311435.162288
230366,97405.0,2018,1.666943e+09,4928.0,338259.508726
230367,97405.0,2019,1.678743e+09,4557.0,368387.842879
230368,97405.0,2020,1.675202e+09,4201.0,398762.726732
230369,97405.0,2021,2.162595e+09,4646.0,465474.586957


In [16]:
# zip code to float
yearly_single_family_by_zip['zip_code'] = yearly_single_family_by_zip['zip_code'].astype(float)
yearly_single_family_by_zip

,zip_code,year,total_sale_value_sum,homes_sold,normalized_sale_price
0,501.0,2012,1080000.0,6.0,180000.0
1,501.0,2013,1530000.0,6.0,255000.0
2,501.0,2015,2465240.0,10.0,246524.0
3,501.0,2016,412000.0,2.0,206000.0
4,501.0,2017,621468.0,4.0,155367.0
...,...,...,...,...,...
237993,99725.0,2016,1350000.0,6.0,225000.0
237994,99725.0,2018,720000.0,6.0,120000.0
237995,99725.0,2020,388800.0,6.0,64800.0
237996,99725.0,2021,882000.0,6.0,147000.0


In [17]:
# seeing what this looks like in a zip with no data - this zip exists in the HPI data we are about to look at next
test = yearly_single_family_by_zip[yearly_single_family_by_zip['zip_code']==99901]
test

,zip_code,year,total_sale_value_sum,homes_sold,normalized_sale_price


In [18]:
# get the values in 2012. i will need this to merge into the HPI data and back calculate estimated prices for years before 2012
yearly_single_family_by_zip_2012 = yearly_single_family_by_zip[yearly_single_family_by_zip['year']==2012]
yearly_single_family_by_zip_2012=yearly_single_family_by_zip_2012.rename(columns={'normalized_sale_price':'normalized_sale_price_2012'})
yearly_single_family_by_zip_2012


,zip_code,year,total_sale_value_sum,homes_sold,normalized_sale_price_2012
0,501.0,2012,1080000.0,6.0,180000.000000
8,1001.0,2012,126476200.0,774.0,163405.943152
20,1002.0,2012,278515300.0,1012.0,275212.747036
32,1003.0,2012,1407000.0,3.0,469000.000000
39,1005.0,2012,21234300.0,163.0,130271.779141
...,...,...,...,...,...
237932,99701.0,2012,66309300.0,377.0,175886.737401
237944,99705.0,2012,175618774.0,800.0,219523.467500
237956,99709.0,2012,173860450.0,756.0,229974.140212
237968,99712.0,2012,86454762.0,355.0,243534.540845


### Section 2: merge the prices into the HPI data going back as far as the HPI data exists
remember, redfin only goes to 2012


In [19]:
# read in the house price index by zip code (this is yearly not quarterly)
hpi_zip5_df = pd.read_excel('data/HPI_AT_BDL_ZIP5.xlsx', skiprows=6, sheet_name='ZIP5')
#hpi_zip5_df.head()

In [20]:
hpi_zip5_df.head(5)

,Five-Digit ZIP Code,Year,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base
0,1001,1984,.,100.00,53.37,52.31
1,1001,1985,16.00,116.00,61.91,60.68
2,1001,1986,14.21,132.48,70.70,69.30
3,1001,1987,21.08,160.41,85.61,83.90
4,1001,1988,17.63,188.68,100.69,98.69


In [21]:
#problem: there are some '.' strings in the HPI column. i replace them with a value i can convert later

# replacing string '.' with a value i can find later as i make everything into floats later on
hpi_zip5_df['HPI'] = hpi_zip5_df['HPI'].replace('.', '-1')


In [22]:
# Filter the HPI data to include only the rows with year 2012
df_2012 = hpi_zip5_df[hpi_zip5_df['Year'] == 2012]
df_2012['HPI'] = df_2012['HPI'].astype(float)
# Find the HPI value for 2012 for each zip code, and I have set the 2012 base at 100 for every zip
df_2012['HPI with 2012 base'] = 100
df_2012['HPI from 2012'] = df_2012['HPI']
df_2012

/var/folders/hl/p3c4xh5x4nv00kvc7sckx1040000gn/T/ipykernel_72560/1675495260.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2012['HPI'] = df_2012['HPI'].astype(float)
/var/folders/hl/p3c4xh5x4nv00kvc7sckx1040000gn/T/ipykernel_72560/1675495260.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2012['HPI with 2012 base'] = 100
/var/folders/hl/p3c4xh5x4nv00kvc7sckx1040000gn/T/ipykernel_72560/1675495260.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


,Five-Digit ZIP Code,Year,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base,HPI with 2012 base,HPI from 2012
28,1001,2012,-4.40,279.95,149.40,146.43,100,279.95
68,1002,2012,-2.37,348.54,184.84,153.00,100,348.54
99,1005,2012,-4.70,150.95,.,122.34,100,150.95
137,1007,2012,-1.67,262.24,164.68,149.66,100,262.24
158,1008,2012,-0.03,122.54,.,.,100,122.54
...,...,...,...,...,...,...,...,...
630422,99712,2012,-0.03,152.05,202.00,138.88,100,152.05
630466,99801,2012,2.80,353.65,254.27,152.90,100,353.65
630491,99824,2012,4.83,156.12,.,138.38,100,156.12
630529,99835,2012,2.80,298.33,276.46,157.45,100,298.33


In [23]:
# merge the 2012 value onto the full zip hpi data
hpi_zip5_df_merged = pd.merge(hpi_zip5_df, df_2012[['Five-Digit ZIP Code', 'HPI from 2012']], on= 'Five-Digit ZIP Code')

In [24]:
hpi_zip5_df_merged.head(10)

,Five-Digit ZIP Code,Year,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base,HPI from 2012
0,1001,1984,.,100.00,53.37,52.31,279.95
1,1001,1985,16.00,116.00,61.91,60.68,279.95
2,1001,1986,14.21,132.48,70.70,69.30,279.95
3,1001,1987,21.08,160.41,85.61,83.90,279.95
4,1001,1988,17.63,188.68,100.69,98.69,279.95
5,1001,1989,0.90,190.37,101.60,99.58,279.95
6,1001,1990,-1.57,187.38,100.00,98.01,279.95
7,1001,1991,-5.67,176.76,94.33,92.46,279.95
8,1001,1992,-1.26,174.54,93.15,91.30,279.95
9,1001,1993,-0.08,174.40,93.07,91.22,279.95


In [25]:
# now calculate hpi based on 2012 for every row
hpi_zip5_df_merged['HPI']=hpi_zip5_df_merged['HPI'].astype(float)
hpi_zip5_df_merged['HPI with 2012 base'] = hpi_zip5_df_merged['HPI']/hpi_zip5_df_merged['HPI from 2012']


In [26]:
# convert the zip code to float for merging later
hpi_zip5_df_merged['Five-Digit ZIP Code']=hpi_zip5_df_merged['Five-Digit ZIP Code'].astype(float)
hpi_zip5_df_merged.head(10)

,Five-Digit ZIP Code,Year,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base,HPI from 2012,HPI with 2012 base
0,1001.0,1984,.,100.00,53.37,52.31,279.95,0.357207
1,1001.0,1985,16.00,116.00,61.91,60.68,279.95,0.414360
2,1001.0,1986,14.21,132.48,70.70,69.30,279.95,0.473227
3,1001.0,1987,21.08,160.41,85.61,83.90,279.95,0.572995
4,1001.0,1988,17.63,188.68,100.69,98.69,279.95,0.673977
5,1001.0,1989,0.90,190.37,101.60,99.58,279.95,0.680014
6,1001.0,1990,-1.57,187.38,100.00,98.01,279.95,0.669334
7,1001.0,1991,-5.67,176.76,94.33,92.46,279.95,0.631398
8,1001.0,1992,-1.26,174.54,93.15,91.30,279.95,0.623468
9,1001.0,1993,-0.08,174.40,93.07,91.22,279.95,0.622968


In [27]:
#merged_prices_hpi = pd.merge(hpi_zip5_df_merged, yearly_single_family_by_zip[['zip_code', 'weighted_mean_sale_price']], left_on='Five-Digit ZIP Code', right_on='zip_code')
merged_prices_hpi = hpi_zip5_df_merged.merge(yearly_single_family_by_zip[['zip_code', 'year', 'normalized_sale_price']], left_on=['Five-Digit ZIP Code', 'Year'], right_on=['zip_code', 'year'], how='left')
merged_prices_hpi['normalized_sale_price'] = round(merged_prices_hpi['normalized_sale_price'],0)
merged_prices_hpi


,Five-Digit ZIP Code,Year,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base,HPI from 2012,HPI with 2012 base,zip_code,year,normalized_sale_price
0,1001.0,1984,.,100.00,53.37,52.31,279.95,0.357207,NaN,NaN,NaN
1,1001.0,1985,16.00,116.00,61.91,60.68,279.95,0.414360,NaN,NaN,NaN
2,1001.0,1986,14.21,132.48,70.70,69.30,279.95,0.473227,NaN,NaN,NaN
3,1001.0,1987,21.08,160.41,85.61,83.90,279.95,0.572995,NaN,NaN,NaN
4,1001.0,1988,17.63,188.68,100.69,98.69,279.95,0.673977,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
630295,99901.0,2018,5.43,271.56,252.08,182.19,226.83,1.197196,NaN,NaN,NaN
630296,99901.0,2019,7.66,292.37,271.39,196.15,226.83,1.288939,NaN,NaN,NaN
630297,99901.0,2020,0.38,293.49,272.43,196.90,226.83,1.293876,NaN,NaN,NaN
630298,99901.0,2021,5.50,309.63,287.42,207.73,226.83,1.365031,NaN,NaN,NaN


In [28]:
# drop excess columns
merged_prices_hpi = merged_prices_hpi.drop(['zip_code', 'year', 'HPI with 2000 base', 'HPI with 1990 base'], axis = 1)

In [29]:
# backfill the empty NaN values in normalized sale price where the year is < 2012 and we don't have price data
# we use HPI with 2012 base * the 2012 normalized sale price value
# first i have to left join the 2012 prices into a new column
#merged_prices_hpi.head()
#yearly_single_family_by_zip_2012.head()
merged_prices_hpi = merged_prices_hpi.merge(yearly_single_family_by_zip_2012[['zip_code','normalized_sale_price_2012']], left_on=['Five-Digit ZIP Code'], right_on=['zip_code'], how='left')
merged_prices_hpi.head()

,Five-Digit ZIP Code,Year,Annual Change (%),HPI,HPI from 2012,HPI with 2012 base,normalized_sale_price,zip_code,normalized_sale_price_2012
0,1001.0,1984,.,100.00,279.95,0.357207,NaN,1001.0,163405.943152
1,1001.0,1985,16.00,116.00,279.95,0.414360,NaN,1001.0,163405.943152
2,1001.0,1986,14.21,132.48,279.95,0.473227,NaN,1001.0,163405.943152
3,1001.0,1987,21.08,160.41,279.95,0.572995,NaN,1001.0,163405.943152
4,1001.0,1988,17.63,188.68,279.95,0.673977,NaN,1001.0,163405.943152


In [30]:
# drop excess zip code column
merged_prices_hpi.drop(['zip_code'], axis = 1)

,Five-Digit ZIP Code,Year,Annual Change (%),HPI,HPI from 2012,HPI with 2012 base,normalized_sale_price,normalized_sale_price_2012
0,1001.0,1984,.,100.00,279.95,0.357207,NaN,163405.943152
1,1001.0,1985,16.00,116.00,279.95,0.414360,NaN,163405.943152
2,1001.0,1986,14.21,132.48,279.95,0.473227,NaN,163405.943152
3,1001.0,1987,21.08,160.41,279.95,0.572995,NaN,163405.943152
4,1001.0,1988,17.63,188.68,279.95,0.673977,NaN,163405.943152
...,...,...,...,...,...,...,...,...
630295,99901.0,2018,5.43,271.56,226.83,1.197196,NaN,NaN
630296,99901.0,2019,7.66,292.37,226.83,1.288939,NaN,NaN
630297,99901.0,2020,0.38,293.49,226.83,1.293876,NaN,NaN
630298,99901.0,2021,5.50,309.63,226.83,1.365031,NaN,NaN


In [31]:
# drop where i don't have sale price value from 2012 to use (no redfin data)
merged_prices_hpi_clean = merged_prices_hpi.drop(merged_prices_hpi[merged_prices_hpi['normalized_sale_price_2012'].isna()].index)
merged_prices_hpi_clean

,Five-Digit ZIP Code,Year,Annual Change (%),HPI,HPI from 2012,HPI with 2012 base,normalized_sale_price,zip_code,normalized_sale_price_2012
0,1001.0,1984,.,100.00,279.95,0.357207,NaN,1001.0,163405.943152
1,1001.0,1985,16.00,116.00,279.95,0.414360,NaN,1001.0,163405.943152
2,1001.0,1986,14.21,132.48,279.95,0.473227,NaN,1001.0,163405.943152
3,1001.0,1987,21.08,160.41,279.95,0.572995,NaN,1001.0,163405.943152
4,1001.0,1988,17.63,188.68,279.95,0.673977,NaN,1001.0,163405.943152
...,...,...,...,...,...,...,...,...,...
630150,99712.0,2018,5.54,171.64,152.05,1.128839,275233.0,99712.0,243534.540845
630151,99712.0,2019,-0.28,171.16,152.05,1.125682,288851.0,99712.0,243534.540845
630152,99712.0,2020,1.73,174.12,152.05,1.145150,241777.0,99712.0,243534.540845
630153,99712.0,2021,7.93,187.92,152.05,1.235909,285384.0,99712.0,243534.540845


In [32]:
# iterate through the rows and calculate the normalized sale price for where i don't have redfin data (which occurs for rows before 2012)
# i just use the hpi for the year based on 2012 values, and the 2012 price for that zip code, to get the fill-in sale price
import numpy as np

for index, row in merged_prices_hpi_clean.iterrows():
    normalized_sale_price=row['normalized_sale_price']
    if pd.isna(normalized_sale_price):
        result = merged_prices_hpi_clean.at[index, 'HPI with 2012 base'] * merged_prices_hpi_clean.at[index, 'normalized_sale_price_2012'] 
        merged_prices_hpi_clean.at[index, 'normalized_sale_price'] = result



In [33]:
merged_prices_hpi_clean

,Five-Digit ZIP Code,Year,Annual Change (%),HPI,HPI from 2012,HPI with 2012 base,normalized_sale_price,zip_code,normalized_sale_price_2012
0,1001.0,1984,.,100.00,279.95,0.357207,58369.688570,1001.0,163405.943152
1,1001.0,1985,16.00,116.00,279.95,0.414360,67708.838742,1001.0,163405.943152
2,1001.0,1986,14.21,132.48,279.95,0.473227,77328.163418,1001.0,163405.943152
3,1001.0,1987,21.08,160.41,279.95,0.572995,93630.817436,1001.0,163405.943152
4,1001.0,1988,17.63,188.68,279.95,0.673977,110131.928394,1001.0,163405.943152
...,...,...,...,...,...,...,...,...,...
630150,99712.0,2018,5.54,171.64,152.05,1.128839,275233.000000,99712.0,243534.540845
630151,99712.0,2019,-0.28,171.16,152.05,1.125682,288851.000000,99712.0,243534.540845
630152,99712.0,2020,1.73,174.12,152.05,1.145150,241777.000000,99712.0,243534.540845
630153,99712.0,2021,7.93,187.92,152.05,1.235909,285384.000000,99712.0,243534.540845


In [34]:
merged_prices_hpi_clean.to_csv('data/cleaned data/cleaned_hpi_price_data.csv', index = False)

### Section 3: Recession data


In [35]:
recession_df = pd.read_csv("data/recessionflag.csv")
recession_df['DATE'] = pd.to_datetime(recession_df['DATE'])
recession_df['year'] = recession_df['DATE'].dt.year
recession_df['quarter'] = recession_df['DATE'].dt.quarter
recession_df = recession_df.rename(columns={'JHDUSRGDPBR':'recession_flag'})
recession_df.head()

,DATE,recession_flag,year,quarter
0,1967-10-01,0.0,1967,4
1,1968-01-01,0.0,1968,1
2,1968-04-01,0.0,1968,2
3,1968-07-01,0.0,1968,3
4,1968-10-01,0.0,1968,4


In [36]:
recession_df.to_csv('data/cleaned data/recession_flag_clean.csv', index = False)